In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter 
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'MSEDE BELA LKGGS )/TSCA(ROALQNTAN',
 'calculationPrice': 'tops',
 'open': 0,
 'openTime': None,
 'openSource': 'lcfaifoi',
 'close': 0,
 'closeTime': None,
 'closeSource': 'ilfafcio',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 148.4,
 'latestSource': 'IEX real time price',
 'latestTime': '1:11:11 PM',
 'latestUpdate': 1667662737214,
 'latestVolume': None,
 'iexRealtimePrice': 148.4,
 'iexRealtimeSize': 103,
 'iexLastUpdated': 1694221147680,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 147.1,
 'previousVolume': 79686625,
 'change': 2.4,
 'changePercent': 0.01612,
 'volume': None,
 'iexMarketPercent': 0.013156507701711178,
 'iexVolume': 737961,
 'avgTotalVo

In [6]:
data['latestPrice']

148.4

In [7]:
data['marketCap']

2535411521173

In [8]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [9]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,148.4,2535411521173,N/A


In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)


In [11]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,152.430,47001915251,N/A
1,AAL,20.990,13490937578,N/A
2,AAP,213.500,14164568808,N/A
3,AAPL,149.020,2513294794490,N/A
4,ABBV,123.020,212623742719,N/A
...,...,...,...,...
500,YUM,119.040,35426609607,N/A
501,ZBH,163.970,34011483056,N/A
502,ZBRA,536.750,29892483590,N/A
503,ZION,52.650,8599777189,N/A


In [18]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [19]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,150.36,46593141292,N/A
1,AAL,20.77,13475529846,N/A
2,AAP,214.41,14052758188,N/A
3,AAPL,151.68,2547146626698,N/A
4,ABBV,122.14,214166616395,N/A
...,...,...,...,...
500,YUM,118.45,34924105902,N/A
501,ZBH,158.05,33801515452,N/A
502,ZBRA,537.54,29803860322,N/A
503,ZION,51.80,8474504424,N/A


In [21]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:100000000


In [22]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,150.36,46593141292,1316
1,AAL,20.77,13475529846,9533
2,AAP,214.41,14052758188,923
3,AAPL,151.68,2547146626698,1305
4,ABBV,122.14,214166616395,1621
...,...,...,...,...
500,YUM,118.45,34924105902,1671
501,ZBH,158.05,33801515452,1252
502,ZBRA,537.54,29803860322,368
503,ZION,51.80,8474504424,3822


In [23]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [26]:
writer.save()